# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)

## İş Problemi (Business Problem)

### FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.  
Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranış öbeklenmelerine göre gruplar oluşturulacak.

## Veri seti son alışverişlerini 2020 - 2021 yıllarında OmniChannel(hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

 **master_id**: Eşsiz müşteri numarası  
 **order_channel** : Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)  
 **last_order_channel** : En son alışverişin yapıldığı kanal  
 **first_order_date** : Müşterinin yaptığı ilk alışveriş tarihi  
 **last_order_date** : Müşterinin yaptığı son alışveriş tarihi  
 **last_order_date_online** : Muşterinin online platformda yaptığı son alışveriş tarihi  
 **last_order_date_offline** : Muşterinin offline platformda yaptığı son alışveriş tarihi  
 **order_num_total_ever_online** : Müşterinin online platformda yaptığı toplam alışveriş sayısı  
 **order_num_total_ever_offline** : Müşterinin offline'da yaptığı toplam alışveriş sayısı  
 **customer_value_total_ever_offline** : Müşterinin offline alışverişlerinde ödediği toplam ücret  
 **customer_value_total_ever_online** : Müşterinin online alışverişlerinde ödediği toplam ücret  
 **interested_in_categories_12** : Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

In [3]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

df_ = pd.read_csv("/Users/umutaykanat/Desktop/ÇALIŞMALAR/Miuul/CRM/FLOMusteriSegmentasyonu/flo_data_20k.csv")
df = df_.copy()

In [4]:
# 2. Veri setinde
        # a. İlk 10 gözlem,
        # b. Değişken isimleri,
        # c. Boyut,
        # d. Betimsel istatistik,
        # e. Boş değer,
        # f. Değişken tipleri, incelemesi yapınız.
df.head(10)
df.columns
df.describe().T # numerik değişkenlere dair istatistikler
df.isnull().sum() # boş değer bulunmamaktadır.
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [5]:
# 3. Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını ifade etmektedir.
# Herbir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [6]:
# 4. Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [7]:
# 5. Alışveriş kanallarındaki müşteri sayısının, toplam alınan ürün sayısı ve toplam harcamaların dağılımına bakınız. 
df.groupby("order_channel").agg({"master_id": "count",
                                "order_num_total": "sum",
                                "customer_value_total": "sum"})

,master_id,order_num_total,customer_value_total
order_channel,,,
Android App,9495,52269.000,7819062.760
Desktop,2735,10920.000,1610321.460
Ios App,2833,15351.000,2525999.930
Mobile,4882,21679.000,3028183.160


In [8]:
# 6. En fazla kazancı getiren ilk 10 müşteriyi sıralayınız.
df.sort_values("customer_value_total",ascending=False)[:10]["master_id"]

11150    5d1c466a-9cfd-11e9-9897-000d3a38a36f
4315     d5ef8058-a5c6-11e9-a2fc-000d3a38a36f
7613     73fd19aa-9e37-11e9-9897-000d3a38a36f
13880    7137a5c0-7aad-11ea-8f20-000d3a38a36f
9055     47a642fe-975b-11eb-8c2a-000d3a38a36f
7330     a4d534a2-5b1b-11eb-8dbd-000d3a38a36f
8068     d696c654-2633-11ea-8e1c-000d3a38a36f
163      fef57ffa-aae6-11e9-a2fc-000d3a38a36f
7223     cba59206-9dd1-11e9-9897-000d3a38a36f
18767    fc0ce7a4-9d87-11e9-9897-000d3a38a36f
Name: master_id, dtype: object

In [9]:
# 7. En fazla siparişi veren ilk 10 müşteriyi sıralayınız.
df.sort_values("order_num_total", ascending=False)[:10]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.000,2.000,684.970,45220.130,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.000,45905.100
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.000,1.000,49.990,12232.250,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.000,12282.240
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.000,109.000,10239.460,143.980,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.000,10383.440
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.000,87.000,8432.250,139.980,[ERKEK],88.000,8572.230
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.000,81.000,3997.550,242.810,[ERKEK],83.000,4240.360
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.000,1.000,1263.760,32654.340,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.000,33918.100
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.000,74.000,4640.770,543.280,"[KADIN, AKTIFSPOR]",77.000,5184.050
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.000,3.000,292.930,5004.950,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.000,5297.880
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.000,1.000,82.480,16836.090,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.000,16918.570
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.000,4.000,843.680,17599.890,"[ERKEK, KADIN, AKTIFSPOR]",70.000,18443.570


In [10]:
# 8. Veri ön hazırlık sürecini fonksiyonlaştırınız.
def data_prep(dataframe):
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    return df

In [11]:
df = df_.copy()
data_prep(df)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN],5.000,939.370
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.000,2013.550
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]",5.000,585.320
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]",2.000,121.970
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR],2.000,209.980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,Android App,Offline,2019-09-21,2020-07-05,2020-06-05,2020-07-05,1.000,2.000,289.980,111.980,"[ERKEK, AKTIFSPOR]",3.000,401.960
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,Desktop,Desktop,2020-03-01,2020-12-22,2020-12-22,2020-03-01,1.000,1.000,150.480,239.990,[AKTIFSPOR],2.000,390.470
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,Ios App,Ios App,2019-09-11,2021-05-24,2021-05-24,2019-09-11,2.000,1.000,139.980,492.960,[AKTIFSPOR],3.000,632.940
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,Android App,Android App,2019-03-27,2021-02-13,2021-02-13,2021-01-08,1.000,5.000,711.790,297.980,"[ERKEK, AKTIFSPOR]",6.000,1009.770


## GÖREV 2: RFM Metriklerinin Hesaplanması



In [12]:
import datetime as dt

In [13]:
# Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız
df["last_order_date"].max() # 2021-05-30
analysis_date = dt.datetime(2021,6,1)

In [14]:
# customer_id, recency, frequnecy ve monetary değerlerinin yer aldığı yeni bir rfm dataframe oluşturalım
rfm = pd.DataFrame()
rfm["customer_id"] = df["master_id"]
rfm["recency"] = (analysis_date - df["last_order_date"]).astype('timedelta64[D]')
rfm["frequency"] = df["order_num_total"]
rfm["monetary"] = df["customer_value_total"]

rfm.head()

,customer_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.000,5.000,939.370
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.000,21.000,2013.550
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.000,5.000,585.320
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.000,2.000,121.970
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.000,2.000,209.980


## GÖREV 3: RF ve RFM Skorlarının Hesaplanması (Calculating RF and RFM Scores)

In [15]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])

rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.000,5.000,939.370,3,4,4
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.000,21.000,2013.550,3,5,5
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.000,5.000,585.320,2,4,3
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.000,2.000,121.970,3,1,1
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.000,2.000,209.980,3,1,1


In [16]:
# recency_score ve frequency_score’u tek bir değişken olarak ifade edilmesi ve RF_SCORE olarak kaydedilmesi
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

# 3. recency_score ve frequency_score ve monetary_score'u tek bir değişken olarak ifade edilmesi ve RFM_SCORE olarak kaydedilmesi
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str))

rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.000,5.000,939.370,3,4,4,34,344
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.000,21.000,2013.550,3,5,5,35,355
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.000,5.000,585.320,2,4,3,24,243
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.000,2.000,121.970,3,1,1,31,311
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.000,2.000,209.980,3,1,1,31,311


## GÖREV 4: RF Skorlarının Segment Olarak Tanımlanması

In [17]:
# Oluşturulan RFM skorların daha açıklanabilir olması için segment tanımlama ve  tanımlanan seg_map yardımı ile RF_SCORE'u segmentlere çevirme
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True) # regular expression ile sınıflandırmaları isimlendirdik.

rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.000,5.000,939.370,3,4,4,34,344,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.000,21.000,2013.550,3,5,5,35,355,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.000,5.000,585.320,2,4,3,24,243,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.000,2.000,121.970,3,1,1,31,311,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.000,2.000,209.980,3,1,1,31,311,about_to_sleep


### 1. Segmentlerin recency, frequnecy ve monetary ortalamalarını inceleyiniz.

In [44]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean","count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep      113.785  1629     2.401  1629  359.009  1629
at_Risk             241.607  3131     4.472  3131  646.610  3131
cant_loose          235.444  1200    10.698  1200 1474.468  1200
champions            17.107  1932     8.934  1932 1406.625  1932
hibernating         247.950  3604     2.394  3604  366.267  3604
loyal_customers      82.595  3361     8.375  3361 1216.819  3361
need_attention      113.829   823     3.728   823  562.143   823
new_customers        17.918   680     2.000   680  339.956   680
potential_loyalists  37.156  2938     3.304  2938  533.184  2938
promising            58.921   647     2.000   647  335.673   647

### 2. RFM analizi yardımı ile 2 case için ilgili profildeki müşterileri bulunuz ve müşteri id'lerini csv ye kaydediniz.

#### a. FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri tercihlerinin üstünde. Bu nedenle markanın  tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak iletişime geçilmek isteniliyor. Bu müşterilerin sadık  ve kadın kategorisinden alışveriş yapan kişiler olması planlandı. Müşterilerin id numaralarını csv dosyasına yeni_marka_hedef_müşteri_id.cvs olarak kaydediniz.
#### b. Erkek ve Çoçuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte iyi müşterilerden olan ama uzun süredir alışveriş yapmayan ve yeni gelen müşteriler özel olarak hedef alınmak isteniliyor. Uygun profildeki müşterilerin id'lerini csv dosyasına indirim_hedef_müşteri_ids.csv olarak kaydediniz.

Sadık kişiler filtrelemesini oluşturduğum *rfm* dataframeinden elde edeceğim. Daha sonra bu id'lere sahip kadın müşterileri *df* adlı dataframe'den filtreleyeceğim.

In [25]:
rfm["segment"].unique()

array(['loyal_customers', 'at_Risk', 'about_to_sleep',
       'potential_loyalists', 'hibernating', 'champions', 'cant_loose',
       'need_attention', 'promising', 'new_customers'], dtype=object)

In [26]:
target_segments_customer_ids = rfm[rfm["segment"].isin(["champions","loyal_customers"])]["customer_id"]

In [31]:
cust_ids = df[(df["master_id"].isin(target_segments_customer_ids)) & df["interested_in_categories_12"].str.contains("KADIN")]["master_id"]

In [32]:
cust_ids.to_csv("yeni_marka_hedef_müşteri_id.cvs", index=False)

In [33]:
cust_ids.shape # hedeflenen müşteri kitlesinde 2497 adet müşteri bulunmaktadır.

(2497,)

In [35]:
target_segments_customes_ids2 = rfm[rfm["segment"].isin(["at_Risk","hibernating","new_customers","cant_loose"])]["customer_id"]

In [39]:
cust_ids2 = df[df["master_id"].isin(target_segments_customes_ids2) & df["interested_in_categories_12"].str.contains("ERKEK")|df["interested_in_categories_12"].str.contains("COCUK")]["master_id"]

In [42]:
df[df["master_id"].isin(target_segments_customes_ids2) & df["interested_in_categories_12"].str.contains("ERKEK")|df["interested_in_categories_12"].str.contains("COCUK")]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.000,2013.550
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]",5.000,585.320
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]",2.000,121.970
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.000,1.000,49.990,113.640,[COCUK],2.000,163.630
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.000,2.000,120.480,934.210,"[ERKEK, COCUK, KADIN]",5.000,1054.690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19934,9777eb76-bed4-11ea-958c-000d3a38a36f,Desktop,Offline,2020-07-05,2020-12-02,2020-11-28,2020-12-02,1.000,2.000,390.960,143.990,"[ERKEK, KADIN]",3.000,534.950
19936,1982ac0e-9f4c-11e9-9897-000d3a38a36f,Android App,Android App,2019-02-08,2020-06-28,2020-06-28,2019-09-28,10.000,2.000,149.980,1468.240,[AKTIFCOCUK],12.000,1618.220
19937,515ca2d8-afdc-11e9-9757-000d3a38a36f,Ios App,Ios App,2019-03-10,2020-08-27,2020-08-27,2019-09-04,5.000,1.000,39.990,604.880,"[COCUK, KADIN]",6.000,644.870
19938,2427ef66-a410-11e9-a2fc-000d3a38a36f,Android App,Android App,2019-04-12,2020-12-14,2020-12-14,2020-06-21,1.000,3.000,580.710,279.990,"[ERKEK, AKTIFSPOR]",4.000,860.700


In [41]:
cust_ids2.to_csv("indirim_hedef_müşteri_ids.csv", index=False)

In [36]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN],5.000,939.370
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.000,2013.550
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]",5.000,585.320
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]",2.000,121.970
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR],2.000,209.980


In [45]:
rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.000,5.000,939.370,3,4,4,34,344,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.000,21.000,2013.550,3,5,5,35,355,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.000,5.000,585.320,2,4,3,24,243,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.000,2.000,121.970,3,1,1,31,311,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.000,2.000,209.980,3,1,1,31,311,about_to_sleep


📌 Proje Çıktıları, Bulgular ve Kazanımlar  
🔍 Proje Özeti

Bu projede müşteri davranışlarını daha iyi anlamak ve aksiyon alınabilir segmentler oluşturmak amacıyla RFM (Recency, Frequency, Monetary) analizi uygulanmıştır. Çalışma boyunca veri temizleme, özellik mühendisliği, segmentasyon ve iş odaklı yorumlama adımları uçtan uca ele alınmıştır.

📊 Temel Bulgular

Müşteriler, satın alma zamanı, sıklığı ve harcama tutarına göre anlamlı segmentlere ayrılmıştır.

Champions ve Loyal Customers gibi yüksek değerli müşteri grupları net şekilde ayrıştırılmıştır.

At Risk ve Hibernating segmentleri, elde tutma (retention) ve geri kazanım (re-activation) çalışmaları için kritik olarak belirlenmiştir.

Segment bazlı analizler, tekil müşteri skorlarından daha güçlü içgörüler sunmuştur.

🎯 İş Değeri ve Aksiyon Önerileri

Yüksek değerli segmentler için kişiselleştirilmiş kampanyalar ve sadakat programları önerilmektedir.

Düşüşte olan segmentler için hedefli indirimler ve hatırlatma kampanyaları planlanabilir.

Pazarlama bütçesinin, segment bazlı optimize edilmesi mümkün hale gelmiştir.

### TEŞEKKÜRLER  
## UMUT AYKANAT